In [3]:
# 환경설정
from dotenv import load_dotenv
load_dotenv()

True

### 요약봇 (Agent)
- 회의록 요약 기능과 이메일 요약 기능을 함수로 작성 > Tools로 생성
    - 서로 다른 프롬프트로 LLM 질의 > 응답 반환
    - `langchain.tools` 패키지의 `Tool` 활용
- agent를 만들어 요청을 처리할 수 있도록 할 것

In [4]:
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

In [5]:
# LLM
model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.2
)

email_data = '''
제목: 테크노바이오 주식회사 면접 일정 안내 - 김지원님

안녕하세요, 김지원님,

테크노바이오 주식회사의 AI 연구개발팀 선임연구원 포지션에 지원해 주셔서 감사합니다. 귀하의 이력서와 지원서를 검토한 결과, 면접을 진행하고자 합니다.

## 면접 일정 안내
- **일시**: 2025년 3월 20일 (목요일) 오후 2:00 - 3:30 (KST)
- **형식**: 대면 면접
- **장소**: 서울특별시 강남구 테헤란로 123 테크노바이오빌딩 8층 회의실 C
- **면접관**: 박현우 연구소장, 이미래 AI팀 팀장

## 면접 준비사항
1. 신분증을 지참해 주세요.
2. 포트폴리오나 작업 샘플이 있으시면 지참하시기 바랍니다.
3. 화상 면접의 경우, 면접 15분 전에 접속하여 시스템 테스트를 권장합니다.

## 면접 형식 및 내용
- 기술 면접 (45분): 머신러닝 알고리즘, 딥러닝 프레임워크, 바이오데이터 분석 역량 관련 질문
- 행동 면접 (30분): 과거 연구 프로젝트 경험 및 팀 협업 사례 중심 질문
- 질의응답 (15분): 회사와 직무에 대한 귀하의 질문

## 기타 안내사항
- 주차가 필요하신 경우, 사전에 알려주시면 주차권을 준비해 드리겠습니다.
- 면접 날짜 변경이 필요하시면 최소 48시간 전에 알려주시기 바랍니다.
- 추가 질문이나 특별한 요청사항이 있으시면 언제든지 문의해 주세요.

면접 확정을 위해 이 이메일에 회신하여 참석 여부를 알려주시기 바랍니다. 김지원님과의 만남을 기대합니다.

감사합니다.

정소연
인사팀 채용담당자
테크노바이오 주식회사
02-123-4567
recruit@technobio.co.kr
'''
meeting_data = '''
# 테크노바이오 주식회사 - AI 연구개발팀 선임연구원 면접 스크립트

**일시**: 2025년 3월 20일 (목요일) 오후 2:00 - 3:30 (KST)  
**장소**: 서울특별시 강남구 테헤란로 123 테크노바이오빌딩 8층 회의실 C  
**면접관**: 박현우 연구소장, 이미래 AI팀 팀장  
**지원자**: 김지원 (AI 연구개발팀 선임연구원 지원자)

---

## 인사 및 소개 (5분)
**이미래**: 안녕하세요, 김지원 님. 테크노바이오에 관심을 가져주셔서 감사합니다. 저는 AI팀 팀장 이미래입니다.
**박현우**: 안녕하세요, 저는 연구소장 박현우입니다. 오늘 시간 내주셔서 감사합니다.
**이미래**: 오늘 면접은 약 90분 정도 진행될 예정입니다. 먼저 기술적인 부분에 대해 45분, 그 다음 경험과 관련된 질문에 30분, 마지막으로 질의응답 시간 15분이 주어집니다. 중간에 질문이 있으시면 언제든지 말씀해 주세요.
**김지원**: 네, 안녕하세요. 기회를 주셔서 감사합니다.

---

## 기술 면접 (45분)
**박현우**: 먼저 김지원 님의 기술적 배경에 대해 간략히 소개해 주시겠어요?
**김지원**: 네, 저는 서울대학교에서 컴퓨터공학 학사와 생명정보학 석사를 마쳤고, 현재 바이오데이터 분석을 위한 딥러닝 알고리즘 개발에 5년간 종사해 왔습니다. 특히 단백질 구조 예측을 위한 알고리즘 개발과 유전체 데이터 분석에 전문성을 가지고 있습니다.
**이미래**: 인상적인 배경이네요. 현재 우리 팀에서는 대규모 바이오 데이터셋에서 패턴을 식별하기 위한 알고리즘을 개발하고 있는데요, 이와 관련하여 어떤 경험이 있으신가요?
**김지원**: 이전 회사에서 10TB 규모의 유전체 데이터를 분석하기 위한 분산 처리 시스템을 설계하고 구현한 경험이 있습니다. 특히 PyTorch와 TensorFlow를 활용한 커스텀 모델을 개발했고, 이를 통해 처리 시간을 60% 단축시켰습니다.
**박현우**: 실제 문제 상황을 하나 제시해 드릴게요. 불균형한 데이터셋(예: 희귀 질병 케이스가 매우 적은 경우)에서 모델의 정확도를 높이기 위해 어떤 접근 방식을 사용하시겠습니까?
**김지원**: 불균형 데이터의 경우 몇 가지 전략을 사용할 수 있습니다. 먼저 데이터 증강(data augmentation)을 통해 소수 클래스의 샘플을 증가시키거나, SMOTE와 같은 오버샘플링 기법을 사용할 수 있습니다. 또한 가중치 손실 함수를 사용하여 소수 클래스의 오분류에 더 큰 페널티를 부여하는 방법도 효과적입니다. 실제로 희귀 유전병 데이터 분석 시 이러한 방법을 조합하여 F1 점수를 0.72에서 0.85로 향상시킨 경험이 있습니다.
**이미래**: 현재 사용하고 계신 딥러닝 프레임워크와 그 선택 이유는 무엇인가요?
**김지원**: 주로 PyTorch를 사용하고 있습니다. 동적 계산 그래프의 유연성이 실험적 연구에 적합하다고 생각하기 때문입니다. 특히 바이오 데이터처럼 복잡한 구조를 가진 데이터를 다룰 때 커스텀 레이어와 손실 함수를 쉽게 구현할 수 있다는 장점이 있습니다. 다만 프로덕션 환경에서는 TensorFlow와 TF Serving을 활용한 경험도 있습니다.
*(기술 면접은 추가 45분 동안 계속되며, 알고리즘 설계, 코드 최적화, 바이오 데이터 특성에 관한 심층 질문이 이어집니다)*

---

## 행동 면접 (30분)
**박현우**: 이제 김지원 님의 경험에 대해 좀 더 알아보고 싶습니다. 팀 내에서 의견 충돌이 있었던 상황과 이를 어떻게 해결했는지 사례를 들어주실 수 있을까요?
**김지원**: 이전 프로젝트에서 데이터 전처리 방식에 대해 팀원들과 의견 차이가 있었습니다. 저는 더 강력한 정규화가 필요하다고 생각했지만, 다른 데이터 과학자들은 원본 데이터의 특성을 유지해야 한다고 주장했습니다. 이 상황에서 저는 두 가지 접근 방식을 모두 구현하여 결과를 비교하는 A/B 테스트를 제안했습니다. 실험 결과, 특정 유형의 데이터에서는 각 방식이 장단점이 있다는 것을 확인할 수 있었고, 이를 바탕으로 하이브리드 접근법을 개발하여 팀 전체의 합의를 이끌어 냈습니다.
**이미래**: 높은 압박감 속에서 일해야 했던 경험과 그 상황을 어떻게 관리했는지 말씀해 주세요.
**김지원**: 중요한 제약회사와의 협업 프로젝트에서 데이터 파이프라인 오류로 인해 분석이 지연되는 상황이 발생했습니다. 마감일까지 3일밖에 남지 않은 상태였습니다. 저는 먼저 팀을 소집해 문제를 명확히 정의하고, 각자의 역할을 재분배했습니다. 또한 클라이언트에게 상황을 투명하게 공유하고 일부 기능은 다음 업데이트로 미루는 협상을 진행했습니다. 24시간 동안 집중 작업한 끝에 핵심 기능을 복구할 수 있었고, 클라이언트의 신뢰를 유지하면서 프로젝트를 성공적으로 마무리했습니다.
**박현우**: AI 연구 분야에서 어떤 트렌드가 가장 흥미롭다고 생각하시나요? 그리고 그것이 바이오 분야에 어떻게 적용될 수 있을까요?
**김지원**: 저는 현재 자기지도학습(self-supervised learning)의 발전이 바이오 분야에 큰 변화를 가져올 것이라고 생각합니다. 특히 라벨링되지 않은 방대한 양의 바이오 데이터에서 의미 있는 패턴을 발견할 수 있는 잠재력이 있습니다. 예를 들어, AlphaFold와 같은 성공 사례에서 볼 수 있듯이, 자기지도학습은 단백질 구조 예측에 혁명을 가져왔습니다. 저는 이 접근법을 약물 개발 과정에 적용하여 후보 물질 스크리닝 시간을 단축하는 연구를 진행해 보고 싶습니다.
*(행동 면접은 추가 30분 동안 계속되며, 리더십 경험, 실패로부터의 학습,
'''

# 회의록 요약 기능 (함수)
def meeting_summary(meeting_text):
    prompt = """
당신은 회의록을 분석하고 핵심 내용을 간결하게 요약하는 전문가입니다. 다음 회의록을 분석하여 주요 내용, 결정 사항, 그리고 후속 조치를 명확하게 요약해 주세요.

## 출력 형식
회의록 요약은 다음 구조로 작성해 주세요:

### 1. 회의 개요
- 회의명: [회의 제목]
- 일시: [날짜 및 시간]
- 참석자: [참석자 목록]
- 소요 시간: [회의 진행 시간]

### 2. 핵심 요약 (3-5줄)
[전체 회의 내용을 3-5줄로 간결하게 요약]

### 3. 주요 논의 사항
- [주요 논의 주제 1]
    - [세부 내용]
- [주요 논의 주제 2]
    - [세부 내용]

### 4. 결정 사항
- [결정 사항 1]
- [결정 사항 2]

### 5. 액션 아이템
- [담당자 1]: [해야 할 일], [기한]
- [담당자 2]: [해야 할 일], [기한]

### 6. 다음 회의
- 일시: [다음 회의 예정 일시]
- 주제: [다음 회의 주요 주제]

## 요약 지침
1. 객관적인 사실만 포함하고 의견이나 해석은 추가하지 마세요.
2. 중요도에 따라 내용을 우선순위화하세요.
3. 전문 용어나 약어가 사용된 경우 명확히 표기하세요.
4. 회의록에 명시적으로 언급되지 않은 내용은 추론하지 마세요.
5. 모든 결정 사항과 액션 아이템은 담당자와 기한이 명확히 표시되어야 합니다.

## 회의록:
{meeting_data}
"""
    
    return model.invoke(prompt)


# 이메일 요약 기능 (함수)

def email_summary(email_text):
    prompt = """
당신은 이메일을 분석하고 핵심 내용을 간결하게 요약하는 전문가입니다. 다음 이메일을 분석하여 주요 내용, 요청 사항, 그리고 마감일을 명확하게 요약해 주세요.

## 출력 형식
이메일 요약은 다음 구조로 작성해 주세요:

### 1. 이메일 개요
- 보낸 사람: [발신자 이름과 이메일]
- 받는 사람: [수신자 이름과 이메일]
- 제목: [이메일 제목]
- 날짜: [이메일 발송 날짜 및 시간]

### 2. 핵심 요약 (1-3줄)
[전체 이메일 내용을 1-3줄로 간결하게 요약]

### 3. 주요 내용
- [주요 내용 1]
- [주요 내용 2]

### 4. 요청 사항
- [요청 사항 1]
- [요청 사항 2]

### 5. 마감일/중요 날짜
- [요청 사항 1에 대한 마감일]
- [요청 사항 2에 대한 마감일]
- [기타 중요 날짜]

### 6. 필요한 조치
- [필요한 조치 1]
- [필요한 조치 2]

## 요약 지침
1. 객관적인 사실만 포함하고 의견이나 해석은 추가하지 마세요.
2. 중요도에 따라 내용을 우선순위화하세요.
3. 요청 사항이 있는 경우 반드시 강조하고 마감일을 명시하세요.
4. 이메일에 명시적으로 언급되지 않은 내용은 추론하지 마세요.
5. 이메일 내 모든 첨부 파일을 목록화하세요.

## 이메일:
{email_data}
"""
    
    return model.invoke(prompt)

# Tool 생성 > Agent 생성 > 테스트 데이터로 확인

tools = [
    Tool(name='회의 요약', func=meeting_summary, description='긴 회의록 내용을 요약한다.'),
    Tool(name='메일 요약', func=email_summary, description='업무 이메일 내용을 요약한다.'),
]


agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True    
)



/var/folders/kp/5_0zhw410tzf9tkb2lt76sw40000gn/T/ipykernel_3118/3144038671.py:191: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [6]:
# result = agent.invoke('회의 내용 요약: ' + meeting_data)

In [7]:
# print(result)

In [8]:
# result = agent.run('메일 내용 요약: ' + email_data)

### News 매니저
1. 최신 뉴스 검색
    - langchain.utilities 패키지의 `SerpAPIWrapper`
2. 뉴스 요약 및 키워드 분석
3. 사용자 맞춤형 뉴스 추천

In [9]:
# !pip install google-search-results

In [10]:
from langchain.utilities import SerpAPIWrapper
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
import os 

In [11]:
serpapi_key = os.getenv("SERP_API_KEY")
search = SerpAPIWrapper(serpapi_api_key=serpapi_key)
query = "최신 AI 뉴스"
news_text = search.run(query)

In [12]:
model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.2
)


def summary_news(text):
    prompt = """
당신은 뉴스 기사를 빠르게 요약해서 전달해야하는 뉴스 앵커입니다. 뉴스 소식을 보고 핵심 부분만 요약하는 요약글을 작성하세요.

다음 뉴스 기사를 간결하게 요약해주세요. 가장 중요한 사실, 인물, 장소와 사건을 포함하되, 부차적인 세부 사항은 생략해도 됩니다. 요약은 핵심 정보만 담아 3-5문장으로 작성하고, 원문의 중립적인 어조를 유지해주세요.

뉴스 기사:
[{news_text}}]
"""

    return model.invoke(prompt)

tool = [Tool(name='뉴스 요약', func=summary_news, description='긴 뉴스 내용을 요약한다')]
agent = initialize_agent(
    tools=tool,
    llm=model,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [14]:
result = agent.invoke('뉴스 요약 내용: ' + news_text)



> Entering new AgentExecutor chain...
Question: 뉴스 요약 내용을 요약해 주세요.
Thought: 주어진 뉴스 내용을 간결하게 요약할 필요가 있다.
Action:
```
{
  "action": "뉴스 요약",
  "action_input": "유상임 장관은 인공지능 기본법의 규제 조항 최소화와 국가 AI 경쟁력 강화를 목표로 하고 있다고 밝혔다. AWS는 딥시크-R1 완전 관리형 서버리스 모델을 출시했으며, 엔비디아는 자율주행차를 위한 LLM을 개발 중이다. 딥시크는 과다한 트래픽으로 서비스 장애를 겪고 있으며 외부 투자는 받지 않겠다고 밝혔다. 오픈AI는 추론 모델의 '생각'을 읽어 잘못된 행동을 감지할 수 있다고 전했다. 한국은 중국의 AI 챗봇 '딥시크'의 신규 앱 다운로드를 제한하고 있다. 삼성전자는 AI 연구시설을 개소하고 인재 확보에 나서고 있다."
}
```

Observation: content='뉴스 기사를 제공해 주시면, 해당 내용을 요약해 드리겠습니다. 기사 내용을 입력해 주세요.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 124, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': Non

In [15]:
print(result)

{'input': '뉴스 요약 내용: [\' entity_type: video_universal.\', \'유상임 장관, "인공지능 기본법, 규제 조항 최소화하고 국가 AI 경쟁력 강화가 목표" · AWS, 딥시크-R1 완전 관리형 서버리스 모델 출시… · 엔비디아, 자율주행차 위한 LLM과 ...\', \'AI타임스 · 과다한 트래픽으로 서비스 장애 시달리는 딥시크..."그래도 외부 투자는 안 받아" · 오픈AI "추론 모델의 \\\'생각\\\' 읽으면 잘못된 행동도 감지할 수 있어" · MoE ...\', "인공지능 (AI) · 비디오, \'딥페이크 사태\' 그 후, 끝나지 않은 피해자들의 이야기, 방송 길이 10,48 · 한국, 중국의 AI 챗봇 \'딥시크\' 신규 앱 다운로드 제한 · 딥페이크: ...", "2월에 공개된 \'딥리서치\'는 인터넷에서 복잡한 연구 작업을 수행할 수 있도록 돕는다. 오픈AI 리스폰스 API (사진=오픈AI 뉴스룸). 리스폰스 API는 오픈AI ...", \'AI News reports on the latest artificial intelligence news and insights. Explore industry trends from the frontline of AI.\', \'메타AI뉴스는 인공지능 소셜 미디어입니다 비즈니스, 경제, AI기반으로 한 기사 제공.\', \'서비스나우는 자사의 플랫폼에 무브웍스의 프론트엔드 AI 에이전트와 엔터프라이즈 검색 기능을 추가할 예정이라고 밝혔다. 모든 워크플로우에서, 모든 ...\', \'(샌프란시스코=연합뉴스) 김태종 특파원 = 미국 경기 침체 우려 등으로 약세를 지속하는 인공지능(AI) 대장주 엔비디아 주가가 11일(현지시간) 반등 ...\', \'삼성전자는 AI의 중요성을 빠르게 인식하고, 주요 국가들에 AI 연구시설을 개소하는 등 적극적인 투자와 인재 확보에 나서고 있다. 회사는 스마트폰, TV, 가전 등 연간 5억 ...\']', 'output': 'Agent stop